In [3]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

1. 얼굴 질병 예측 모델

In [39]:
from pathlib import Path
train_dir = Path(r'D:\main\train')
test_dir = Path(r'D:\main\testing')


In [40]:
Acne = list(train_dir.glob('Acne/*'))
Actinic = list(train_dir.glob('Actinic Keratosis/*'))
Basal = list(train_dir.glob('Basal Cell Carcinoma/*'))
Eczemaa = list(train_dir.glob('Eczemaa/*'))
Rosacea = list(train_dir.glob('Rosacea/*'))


In [41]:
batch_size = 32
img_height = 180
img_width = 180

In [42]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 440 files belonging to 5 classes.
Found 185 files belonging to 5 classes.


In [43]:
class_names = train_ds.class_names

In [44]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [45]:
normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.999292


In [46]:
num_classes = len(class_names)

facemodel = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

c:\Users\mash0\.conda\envs\facedisease\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [47]:
facemodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [49]:
epochs=10
history = facemodel.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 16s 848ms/step - accuracy: 0.1547 - loss: 2.1263 - val_accuracy: 0.2108 - val_loss: 1.5838
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 638ms/step - accuracy: 0.2840 - loss: 1.5507 - val_accuracy: 0.2649 - val_loss: 1.5558
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 675ms/step - accuracy: 0.3745 - loss: 1.4030 - val_accuracy: 0.3459 - val_loss: 1.4838
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 610ms/step - accuracy: 0.4023 - loss: 1.3578 - val_accuracy: 0.4811 - val_loss: 1.3334
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 606ms/step - accuracy: 0.5639 - loss: 1.1015 - val_accuracy: 0.3676 - val_loss: 1.4109
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 653ms/step - accuracy: 0.5761 - loss: 1.0395 - val_accuracy: 0.4973 - val_loss: 1.3024
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 687ms/step - accuracy: 0.6331 - loss: 0.9349 - val_accuracy: 0.5297 - val_loss: 1.2827
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 826ms/step - accuracy: 0.6963 - loss: 0.7803 - val_accuracy: 